This notebook was designed to work with [Google Colab](https://colab.research.google.com/github/lokdoesdata/syracuse-assorted/blob/main/ist_652/homework_2/lok_ngan_homework_2.ipynb).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lokdoesdata/syracuse-assorted/blob/main/ist_652/homework_2/lok_ngan_homework_2.ipynb)

# IST 652 - Homework 2: Semi-structured Data
Lok Ngan

Due: May 28, 2021

-------------
The main outline of this assignment is to show how to read in JSON data from a MongoDB collection or from a file.

## Set Up

### Install Geopandas on Google Colab

In [ ]:
%pip install geopandas

### Import libraries

The library used in this script are:
* `urlopen` from `urllib.request`, which is used to read the json files from the internet
* `Path` from `pathlib`, which is used for I/O
* `json`, which is used to handle json files
* `geopandas`, which is used to read geojson files into a GeoDataFrame

In [ ]:
# Vanilla Python
from urllib.request import urlopen
from pathlib import Path
import json

# Needed Packages
import pandas as pd
import geopandas as gpd
import plotly.express as px

### I/O Path

In [ ]:
DATA_PATH = Path.cwd().joinpath('data')
DATA_PATH.mkdir(parents=True, exist_ok=True)

OUTPUT_PATH = Path.cwd().joinpath('output')
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

## Data

The data used for this assignment came from Esri.  It is the [list of ZIP Code](https://www.arcgis.com/home/item.html?id=1eeaf4bb41314febb990e2e96f7178df), summarized as points, in the United States.  This list of ZIP code includes PO Box and single site ZIP Codes, and where applicable, the population and area of each ZIP Code.

In order to get to this data, the [Esri REST API](https://developers.arcgis.com/rest/) was used to query the data.  The feature server for this data set is located [here](https://services.arcgis.com/P3ePLMYs2RVChkJx/ArcGIS/rest/services/USA_ZIP_Code_Points_analysis/FeatureServer/0/).  The query output format is a JSON file.  The maximum record pull per call is 2,000 records.  A while loop was created to gather all ZIP codes data by evaluating the returned json file for the existance of property 'exceededTransferLimit'.

In [ ]:
def get_usps_zip_code_point():
    """Return Esri USA ZIP Code Points as a GeoDataFrame.

    Author: Lok Ngan (April 25, 2021)
    """
    # Construct URL
    URL = r'https://services.arcgis.com/P3ePLMYs2RVChkJx/ArcGIS/rest/services/'
    URL = URL + r'USA_ZIP_Code_Points_analysis/FeatureServer/0/query?'
    URL = URL + r'f=geojson&outFields=*&where=1=1&resultOffset='

    # OUTPUT FILE goes here
    OUTPUT_FILE = DATA_PATH.joinpath('usps_zip_code_points.geojson')

    # If output file already exist, just read it!
    if OUTPUT_FILE.is_file():
        return(gpd.read_file(OUTPUT_FILE))

    # Esri has a limit of 2,000 per pull on REST, the records can be cycled using the resultOffset parameter
    offset = 0
    out_features = []

    # While exceeding the transfer limit, the offset will continue to go up by 2,000 until all data points are retrieved
    while True:
        temp_url = URL + str(offset*2000)

        with urlopen(temp_url) as resp:
            temp_json = json.load(resp)

        out_features.extend(temp_json['features'])

        if temp_json.get('properties') is None:
            break
        if temp_json.get('properties').get('exceededTransferLimit') is None:
            break

        offset += 1

    # combine all of the features into a dictionary
    out_geojson = dict(type='FeatureCollection', features=out_features)

    # write dictionary into a geojson file
    with open(OUTPUT_FILE, 'w') as out:
        json.dump(out_geojson, out)

    # Read the geojson file
    if OUTPUT_FILE.is_file():
        return(gpd.read_file(OUTPUT_FILE))

    return(out_geojson)


In [ ]:
gdf = get_usps_zip_code_point()

### Dimension

There are 41,080 records in the dataset across 8 columns:

| Header        | Description                               |
| :------------ | :---------------------------------------- |
| OBJECTID      | Esri Index                                |
| ZIP_CODE      | 5-digit USPS ZIP Code                     |
| PO_NAME       | Post Office Name                          |
| STATE         | State                                     |
| ZIP_TYPE      | ZIP Code type, area or single site        |
| POPULATION    | Population of ZIP Code                    |
| SQMI          | Surface Area of ZIP Code in Sq Mi         |
| geometry      | Point geometry (centroid) of the ZIP Code |

In [ ]:
gdf.shape

### Remove Unnecessary Columns

OBJECTID is the only column removed

In [ ]:
gdf.drop('OBJECTID', axis=1, inplace=True)

### Evaluate for Null

Population and SqMi have null values.  This makes sense as some single site ZIP codes would not have an area and would not have population.  They are filled with 0 for the purpose of this analysis.

In [ ]:
for col in gdf.columns:
    print(f"{col} has {sum(pd.isnull(gdf[col]))} null values")

### Fill null with 0

In [ ]:
gdf.fillna(0, inplace=True)

## Analysis

### Which state has the most population?

This can be done by grouping by state, and summing the population.

In [ ]:
gdf_pop = gdf.groupby('STATE').agg({'POPULATION':'sum'})

California, Texas, and Florida has the highest populations.

In [ ]:
gdf_pop.sort_values('POPULATION', ascending=False)

In [ ]:
fig_pop = px.choropleth(
    gdf_pop,
    locations=gdf_pop.index, 
    locationmode='USA-states', 
    color=gdf_pop['POPULATION'], 
    scope='usa',
    labels={'POPULATION': 'Population'}
)

fig_pop.update_layout(
    margin=dict(r=0, t=75, l=0, b=15),
    title_text = '2018 United States Population By States<br><sub>Data Source: Esri</sub>'
)

### Which state has the most ZIP code?

This can be done by grouping by states, and counting the ZIP Codes.  Note that this step can actually be done with along with the first data question by adding another key-value to the aggregation method, but they are separated for demonstration purposes.

In [ ]:
gdf_zip_code = gdf.groupby('STATE').agg({'ZIP_CODE':'count'})

Texas, California, and Pennsylvania have the most ZIP Codes.

In [ ]:
gdf_zip_code.sort_values('ZIP_CODE', ascending=False)

In [ ]:
fig_zip_code = px.choropleth(
    gdf_zip_code,
    locations=gdf_pop.index, 
    locationmode='USA-states', 
    color=gdf_zip_code['ZIP_CODE'], 
    scope='usa',
    labels={'ZIP_CODE': 'ZIP Code'}
)

fig_zip_code.update_layout(
    margin=dict(r=0, t=75, l=0, b=15),
    title_text = 'Number of USPS ZIP Codes in Each State<br><sub>Data Source: Esri</sub>'
)

### Which ZIP code has the highest population density?

Population density will be estimated using population divided by sqmi.  A list comprehension was used for the calculation. 

In [ ]:
# making a deepcopy of the dataframe
gdf_density = gdf.copy()

In [ ]:
gdf_density['pop_density'] = [pop/sqmi if sqmi > 0 else 0 for (pop, sqmi) in zip(gdf_density['POPULATION'], gdf_density['SQMI'])]

Unsurprisingly, NY has the most densely populated ZIP Code in the United States.  The top 25 most densely populated ZIP Codes are all in New York.

In [ ]:
gdf_density.sort_values('pop_density', ascending=False).head(25)

In [ ]:
gdf_pop_density = gdf_density[gdf_density['pop_density']>0].copy()

In [ ]:
fig_density = px.scatter_mapbox(
    gdf_density,
    lat=gdf_density.geometry.y,
    lon=gdf_density.geometry.x,
    hover_name='ZIP_CODE',
    hover_data=['PO_NAME', 'STATE', 'POPULATION', 'SQMI'],
    color='pop_density',
    size='pop_density',
    mapbox_style='carto-positron',
    zoom=2,
    center={'lat': 45, 'lon': -120},
    labels={'pop_density': 'Population<br>Density'}
)


fig_density.update_layout(
    title_text = "United States' Population Density By ZIP Code<br><sub>Data Source: Esri</sub>",
    margin=dict(r=0, t=75, l=0, b=15)
)

In [ ]:
fig.show()